In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import pickle
import torch

In [2]:
import sys
import os

# Add the 'project' directory to the path
sys.path.append(os.path.abspath('..'))

from project_code.data.load_data import load_col_types, load_data
from project_code.data.prepare_data_sklearn import get_features_targets
from project_code.utils.results import get_best_model_file
from project_code.inference.parameters import get_core_parameter_predictions, convert_output_to_parameter_predictions, PARAMETER_COLS

# Loading data and models

In [3]:
datasets_folder = f'../data/processed/'

dataset_of_model = {
    'SRTaxo1NN': 'biologist_no_pub_age',  
    'Taxo1NN': 'biologist_no_pub_age',  
    'RandomForestRegressor': 'final_taxonomy_ecocodes',
    'MultiTaskElasticNet': 'final_taxonomy_ecocodes',
    'MLP': 'final_taxonomy_ecocodes',
    'MLPSC': 'final_taxonomy_ecocodes',
    'DEBNetHC': 'final_taxonomy_ecocodes',
    'DEBNetSC': 'final_taxonomy_ecocodes',
}


## Loading best models

In [4]:
def load_model(model_file, results_folder):
    if model_file[-4:] == '.pkl':
        with open(f"{results_folder}/models/{model_file}", 'rb') as f:
            model = pickle.load(f)
    elif model_file[-4:] == '.pth':
        model = torch.load(f"{results_folder}/models/{model_file}", weights_only=False)
        model.eval()
    return model

In [5]:
best_models = {}
best_models_test_performance_files = {}
for mt in dataset_of_model.keys():  
    results_folder = f'../results/{dataset_of_model[mt]}'
    metric = 'GEF'
    model_file = get_best_model_file(results_folder=results_folder, model_type=mt, metric=metric)
    #print(mt, model_file)
    if model_file is not None:
        best_models[mt] = load_model(model_file, results_folder)
        test_performance_filename =  model_file[:-4] + '.csv'
        best_models_test_performance_files[mt] = os.path.join(results_folder, 'test_performance', test_performance_filename)

print(best_models.keys())

dict_keys(['SRTaxo1NN', 'Taxo1NN', 'MultiTaskElasticNet', 'MLP', 'MLPSC'])


## Loading data

In [6]:
all_dfs = {}
all_col_types = {}
all_data = {}
for dataset_name in list(set(dataset_of_model.values())):
    results_folder = f'../results/{dataset_name}'
    all_dfs[dataset_name] = load_data(dataset_name=dataset_name, data_split='train_test', datasets_folder=datasets_folder)
    all_col_types[dataset_name] = load_col_types(dataset_name=dataset_name, datasets_folder=datasets_folder)
    if 'biologist' not in dataset_name:
        all_data[dataset_name] = get_features_targets(all_dfs[dataset_name], all_col_types[dataset_name])
    else:
        model = best_models['Taxo1NN'] if 'Taxo1NN' in best_models else best_models['SRTaxo1NN']
        encoded_dfs = {}
        # Encoded data with trained model encoders
        for split in ('train', 'test'):
            encoded_dfs[split] = model.regressor.encode_data(all_dfs[dataset_name][split])
        all_data[dataset_name] = get_features_targets(data=encoded_dfs, col_types=all_col_types[dataset_name])

# Visualize predictions

In [7]:
taxonomy_cols = [col for col in all_col_types['final_taxonomy_ecocodes']['input']['all'] if 'class' in col]

hue_series = {
    'metamorphosis': pd.concat([all_dfs['biologist_no_pub_age'][data_split]['metamorphosis'] for data_split in ['train', 'test']]),
    'class': pd.concat([pd.from_dummies(all_dfs['final_taxonomy_ecocodes'][data_split][taxonomy_cols], sep='_') for data_split in ['train', 'test']])['class'],
    'climate': None,
    'habitat': None,
    'migrate': None,
    'food': None
}

hue_orders = {
    'metamorphosis': [False, True],
    'class': None,
    'climate': None,
    'habitat': None,
    'migrate': None,
    'food': None
}


In [10]:
def plot_residuals_df(model_type, plot_kind, data_split, groupby, scale):
    # Get data for the model
    dataset_name = dataset_of_model[model_type]
    data = all_data[dataset_name]
    col_types = all_col_types[dataset_name]
    model = best_models[model_type]

    # Get predictions:
    X = data[data_split]['input']
    y_true_ps = data[data_split]['output']
    if 'DEBNet' in model_type:
        y_pred_ps = model.predict(torch.tensor(X, dtype=torch.float32))
    elif 'Taxo1NN' in model_type and data_split in ['train', 'val']:
        distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, data_split='train')
        y_hat, indices = model.regressor_.get_predictions_from_distance_matrix(distance_matrix)
        y_pred_ps = model.regressor_.apply_scaling_relationships(model.regressor_.train_data, y_hat, indices)
    else:
        y_pred_ps = model.predict(data[data_split]['input'])

    if scale == 'model':
        cols_to_plot = col_types['output']['all']
        target_df = pd.DataFrame(y_true_ps, columns=col_types['output']['all'])
        pred_df = pd.DataFrame(y_pred_ps, columns=col_types['output']['all'])
        
    elif scale == 'parameter':
        cols_to_plot = PARAMETER_COLS            
        target_df = convert_output_to_parameter_predictions(y_true_ps, data[data_split]['input'], col_types)
        pred_df = convert_output_to_parameter_predictions(y_pred_ps, data[data_split]['input'], col_types)

    target_df.set_index(all_dfs[dataset_name][data_split].index, inplace=True)
    pred_df.set_index(all_dfs[dataset_name][data_split].index, inplace=True)
    if plot_kind == 'residual_vs_predicted':
        if scale == 'model':
            residuals_df = target_df - pred_df
        elif scale == 'parameter':
            residuals_df = (target_df - pred_df) / target_df  

    # Plot predictions vs targets  
    n_cols = 3
    n_rows = np.ceil(len(cols_to_plot) / n_cols).astype(int)
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(16, 5*n_rows), tight_layout=True)
    fig.suptitle(model_type, fontsize=16)
    margin_factor = 0.05
    for i, col in enumerate(cols_to_plot):
        ax = axes[i // n_cols, i % n_cols]

        if scale == 'parameter' and col != 'kap':
            plot_log_scale_x = True
            if plot_kind == 'residual_vs_predicted':
                plot_log_scale_y = False
            else:
                plot_log_scale_y = True
        elif scale == 'model' and (col in col_types['output']['log'] or 'Taxo1NN' in model_type):
            plot_log_scale_x = True
            plot_log_scale_y = True
        else:
            plot_log_scale_x = False
            plot_log_scale_y = False
        
        if plot_kind == 'residual_vs_predicted':
            sns.scatterplot(x=pred_df[col], y=residuals_df[col], ax=ax, hue=hue_series[groupby], hue_order=hue_orders[groupby]) # Fix
            min_v = (1-margin_factor)*min(target_df[col].min(), pred_df[col].min())
            max_v = (1+margin_factor)*max(target_df[col].max(), pred_df[col].max())
            ax.set_xlim([min_v, max_v])

            ax.plot([min_v, max_v], [0, 0], 'k--')
            ax.set_ylabel('Residuals (actual - predicted)')

        elif plot_kind == 'actual_vs_predicted':
            sns.scatterplot(x=pred_df[col], y=target_df[col], ax=ax, hue=hue_series[groupby], hue_order=hue_orders[groupby])
            min_v = (1-margin_factor)*min(target_df[col].min(), pred_df[col].min())
            max_v = (1+margin_factor)*max(target_df[col].max(), pred_df[col].max())
            ax.set_xlim([min_v, max_v])
            ax.set_ylim([min_v, max_v])
            ax.plot([min_v, max_v], [min_v, max_v], 'k--')
            ax.set_ylabel('Actual values')

        if plot_log_scale_x:
            ax.set_xscale('log')
        else:
            ax.set_xscale('linear')
        
        if plot_log_scale_y:
            ax.set_yscale('log')
        else:
            ax.set_yscale('linear')

        ax.set_xlabel('Predicted values') 
        #r2 = metrics.r2_score(target_df, pred_df)
        ax.set_title(f"{col}")

model_selector = widgets.Dropdown(options=list(best_models.keys()), value='MLP', description='Model:')
plot_selector = widgets.Dropdown(options=['actual_vs_predicted', 'residual_vs_predicted'], value='actual_vs_predicted', description='Plot Type:')
data_split_selector = widgets.Dropdown(options=['train', 'test'], value='test', description='Data Split: ')
groupby_selector = widgets.Dropdown(options=['metamorphosis', 'class', 'climate', 'habitat', 'migrate', 'food'])
scale_selector = widgets.Dropdown(options=['model', 'parameter',], value='parameter')
widgets.interactive(plot_residuals_df, model_type=model_selector, plot_kind=plot_selector, data_split=data_split_selector, groupby=groupby_selector, scale=scale_selector)

interactive(children=(Dropdown(description='Model:', index=3, options=('SRTaxo1NN', 'Taxo1NN', 'MultiTaskElast…

# Save parameter predictions

### AmP values

In [ ]:
dataset_name = 'biologist_no_pub_age'
dfs = all_dfs[dataset_name]
col_types = all_col_types[dataset_name]
gt_df = pd.concat({ds: dfs[ds][col_types['output']['all']] for ds in ('train', 'test')}).reset_index(level=0, names='data_split')
gt_pars_df = get_core_parameter_predictions(dfs, pred_df=gt_df, col_types=col_types)
#gt_pars_df.to_csv(f'../results/parameter_predictions/AmP_predictions.csv', float_format='%.6e')
gt_pars_df

,data_split,z,p_M,kap,v,E_G,E_Hb,E_Hx,E_Hj,E_Hp,k_J
species,,,,,,,,,,,
Thalassarche_chrysostoma,train,5.440655,572.958112,0.932257,0.017264,7322.175732,8849.132656,9291.589289,8849.132656,2.572881e+05,0.023497
Hynobius_quelpaertensis,train,1.183220,51.098233,0.490739,0.037124,7322.175732,54.904856,57.650099,54.904856,1.132485e+04,0.002000
Astyanax_mexicanus,train,1.718575,49.129132,0.993391,0.022779,5230.125523,0.007697,0.008081,0.014183,1.170900e+02,0.002000
Butastur_rufipennis,train,2.564512,1280.082165,0.929643,0.025569,7322.175732,2698.872220,2833.815831,2698.872220,2.536023e+04,0.052769
Chioglossa_lusitanica,train,1.506129,57.151233,0.613918,0.246785,7322.175732,479.768444,503.756866,479.768444,1.047607e+04,0.002000
...,...,...,...,...,...,...,...,...,...,...,...
Pelophylax_saharicus,test,4.659641,11.619885,0.555177,0.089015,7322.175732,30.489465,32.013938,30.489465,1.542382e+05,0.000471
Rhinobatos_productus,test,16.052416,15.935844,0.862744,0.220510,5230.125523,16687.708052,17522.093455,16687.708052,2.038832e+06,0.002007
Zapteryx_brevirostris,test,8.008856,13.022777,0.643357,0.016723,5230.125523,31142.570861,32699.699404,31142.570861,7.234801e+05,0.002000


### ML models

In [ ]:
def save_parameter_predictions(model_type):
    model = best_models[model_type]
    dataset_name = dataset_of_model[model_type]
    dfs = all_dfs[dataset_name]
    data = all_data[dataset_name]
    col_types = all_col_types[dataset_name]
    pred_df = pd.DataFrame()
    for split in ('train', 'test'):
        if 'Taxo1NN' in model_type and split == 'train':
            train_distance_matrix = model.regressor_._compute_distance_matrix(model.regressor_.train_data, model.regressor_.train_data, data_split='train')
            y_hat, indices = model.regressor_.get_predictions_from_distance_matrix(train_distance_matrix)
            y_pred = model.regressor_.apply_scaling_relationships(model.regressor_.train_data, y_hat, indices)
        else:
            y_pred = model.predict(data[split]['input'])
        split_pred_df = pd.DataFrame(data=y_pred, index=dfs[split].index, columns=col_types['output']['all'])
        split_pred_df['data_split'] = split
        pred_df = pd.concat([pred_df, split_pred_df])
    pars_df = get_core_parameter_predictions(dfs, pred_df=pred_df, col_types=col_types)
    predictions_file_name = f'../results/parameter_predictions/{model_type}_predictions.csv'
    pars_df.to_csv(predictions_file_name, float_format='%.10e')
    print(f'Saved parameter predictions for model {model_type} in {predictions_file_name}')

    return pars_df

In [ ]:
#for model_type in best_models:
    #save_parameter_predictions(model_type)

## Finding the bug in feasibility for TaxonomicKNNRegressor

# Compare parameter predictions

## Test performance

In [11]:
best_models['SRTaxo1NN'].regressor_.weight_factor

0.6880000000000001

In [ ]:
model_list = ['DEBNetHCSoftplus', 'DEBNet', 'DEBNetSC', 'TaxonomicKNNRegressor']
model_list = best_models.keys()
gef_test_perf_df = pd.DataFrame(index=model_list, columns=PARAMETER_COLS)
smape_test_perf_df = pd.DataFrame(index=model_list, columns=PARAMETER_COLS)
for model_type in model_list:
    model_test_df = pd.read_csv(f"{best_models_test_performance_files[model_type]}", index_col=0)
    gef_test_perf_df.loc[model_type, :] = model_test_df.loc[:, 'geometric_error_factor']
    smape_test_perf_df.loc[model_type, :] = model_test_df.loc[:, 'symmetric_mean_absolute_percentage_error']

In [ ]:
gef_test_perf_df

,p_Am,kap,v,p_M,E_Hb,E_Hj,E_Hp,k_J,s_M
SRTaxo1NN,2.591915,1.182316,1.521516,2.784039,7.341612,7.548859,4.525898,1.69101,1.255999
Taxo1NN,2.633952,1.187791,1.518533,2.786616,6.566969,13.82922,4.701839,1.719058,1.984639
MultiTaskElasticNet,2.139491,1.114621,1.464876,2.866744,3.411204,4.225567,2.593713,1.849292,1.821698
MLP,2.067639,1.107662,1.469628,2.880215,3.639013,4.097735,2.320432,1.734107,1.705027
MLPSC,2.035213,1.103422,1.453222,2.795417,3.036245,3.778563,2.248085,1.703921,1.717142


In [ ]:
smape_test_perf_df

,p_Am,kap,v,p_M,E_Hb,E_Hj,E_Hp,k_J,s_M
SRTaxo1NN,35.736742,8.1893,18.938881,36.982674,57.619128,57.474583,51.981813,18.942762,9.466246
Taxo1NN,36.533544,8.410142,18.848835,37.076133,54.798182,62.977394,52.302228,19.532469,28.425247
MultiTaskElasticNet,32.362863,5.31774,17.566134,39.336747,43.591369,47.74033,38.410857,23.633563,27.103449
MLP,30.788195,5.001078,17.469656,39.26646,42.636696,45.416679,34.900654,21.139441,23.632332
MLPSC,30.735752,4.820204,16.986429,38.995155,37.710564,44.234802,33.624695,20.681767,23.751528


In [ ]:
model_labels = {
    'DEBNetHC': 'DEBNet w/ HC',
    'DEBNetSC': 'DEBNet w/ SC',
    'Taxo1NN': 'Taxonomic 1-NN',
    'SRTaxo1NN': 'Taxonomic 1-NN w/ S.R.',
    'MultiTaskElasticNet': 'Elastic Net',
    'RandomForestRegressor': 'Random Forest',
    'MLP': 'MLP',
    'MLPSC': 'MLP w/ SC',
}

In [ ]:
#for mt, row in mape_df.iterrows():
test_metrics_df_dict = {
    'GEF': gef_test_perf_df,
    'sMAPE': smape_test_perf_df,
}
for metric, df in test_metrics_df_dict.items():
    print(f'\n\n{metric} test performance:\n')
    for mt in model_list:
        row = df.loc[mt]
        line = f"{model_labels[mt]} "
        for par in PARAMETER_COLS:
            mape = row[par]
            if mape == min(df[par]):
                line += f' & \\textbf{{ {mape:.4f} }}'
            else:
                line += f' & {mape:.4f}'
        line += ' \\\\'
        print(line)



GEF test performance:

Taxonomic 1-NN w/ S.R.  & 2.5919 & 1.1823 & 1.5215 & \textbf{ 2.7840 } & 7.3416 & 7.5489 & 4.5259 & \textbf{ 1.6910 } & \textbf{ 1.2560 } \\
Taxonomic 1-NN  & 2.6340 & 1.1878 & 1.5185 & 2.7866 & 6.5670 & 13.8292 & 4.7018 & 1.7191 & 1.9846 \\
Elastic Net  & 2.1395 & 1.1146 & 1.4649 & 2.8667 & 3.4112 & 4.2256 & 2.5937 & 1.8493 & 1.8217 \\
MLP  & 2.0676 & 1.1077 & 1.4696 & 2.8802 & 3.6390 & 4.0977 & 2.3204 & 1.7341 & 1.7050 \\
MLP w/ SC  & \textbf{ 2.0352 } & \textbf{ 1.1034 } & \textbf{ 1.4532 } & 2.7954 & \textbf{ 3.0362 } & \textbf{ 3.7786 } & \textbf{ 2.2481 } & 1.7039 & 1.7171 \\


sMAPE test performance:

Taxonomic 1-NN w/ S.R.  & 35.7367 & 8.1893 & 18.9389 & \textbf{ 36.9827 } & 57.6191 & 57.4746 & 51.9818 & \textbf{ 18.9428 } & \textbf{ 9.4662 } \\
Taxonomic 1-NN  & 36.5335 & 8.4101 & 18.8488 & 37.0761 & 54.7982 & 62.9774 & 52.3022 & 19.5325 & 28.4252 \\
Elastic Net  & 32.3629 & 5.3177 & 17.5661 & 39.3367 & 43.5914 & 47.7403 & 38.4109 & 23.6336 & 27.1034 \